In [3]:
from keras_unet_collection import losses
import numpy as np

def custom_focal_tversky(y_true, y_pred, alpha=0.7, gamma=4/3):
    return losses.focal_tversky(y_true, y_pred, alpha=alpha, gamma=gamma)

custom_focal_tversky(masks[0], masks[1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.49051696>

In [11]:
import sys
sys.path.append("../dataset/")

import datetime
import glob
import json
import matplotlib.pyplot as plt
import nrrd
import numpy as np
import os
import tensorflow as tf
import keras_unet_collection

from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
from dataset import UltraSoundImages
from keras_unet_collection import models
from keras_unet_collection import losses
from tensorflow.keras.utils import Sequence
from tensorflow import keras
from tensorflow.keras.preprocessing.image import save_img
from utils import *


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

def custom_focal_tversky(y_true, y_pred, alpha=0.7, gamma=4/3):
    return losses.focal_tversky(y_true, y_pred, alpha=alpha, gamma=gamma)

class LocalTversky(tf.keras.losses.Loss):
    def __init__(self):
        super(LocalTversky, self).__init__()
        
    def call(self, y_true, y_pred):
        return losses.focal_tversky(y_true, y_pred, alpha=0.7, gamma=4/3)

class DiceCoefficient(tf.keras.metrics.Metric):
    def __init__(self, name="dice_coefficient", **kwargs):
        super(DiceCoefficient, self).__init__(name=name, **kwargs)
        self.intersection = self.add_weight(name="intersection", initializer="zeros")
        self.union = self.add_weight(name="union", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        self.intersection.assign_add(tf.reduce_sum(y_true * y_pred))
        self.union.assign_add(tf.reduce_sum(y_true) + tf.reduce_sum(y_pred))

    def result(self):
        return (2.0 * self.intersection + 1e-5) / (self.union + 1e-5)

    def reset_states(self):
        self.intersection.assign(0.0)
        self.union.assign(0.0)
        
        
class ImagesInterCheckpoint(tf.keras.callbacks.Callback):

    def __init__(self, input_image, every_n_epoch=10):
        super(ImagesInterCheckpoint, self).__init__()
        self.every_n_epoch = every_n_epoch
        self.input_image = input_image
        save_img(f'{images_path}input_image.png', self.input_image)
    
    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.every_n_epoch == 0:
            output_mask = self.model.predict(np.expand_dims(self.input_image, axis=0))[0]
            output_mask = (output_mask*255).astype('int')
            # save_img(f'{images_path}pred_{epoch}.png', output_mask)

            
def fix_gpu():
    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)


model_type = 'transunet_2d'
dataset_type = 'samsung'
batch_size = 4
loss_type = 'custom_focal_tversky'
epochs = 5000
patiance = 100
dependency_model = None
custom_directory = 'size_320_model_1'
input_size = 320


augment = True
image_input_shape = (input_size, input_size, 1)
image_gen_shape = (image_input_shape[0], image_input_shape[1], 1)
# filter_num = [64, 128, 256, 512, 1024]
filter_num = [64, 128, 256, 512]

train_gen, val_gen, test_gen = get_dataset(dataset_type, batch_size, image_gen_shape, augment)

output_path = 'output'
output_path = f'{output_path}/{model_type}/{dataset_type}/'
if not os.path.exists(output_path):
    os.mkdir(output_path)

if custom_directory:
    directory = custom_directory
elif args['test']:
    directory = 'test'
else:
    directory = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')

output_path = f'{output_path}{directory}/'
images_path = f'{output_path}/images/' 
meta_path = f'{output_path}/meta/'
model_path = f'{output_path}/model/'
checkpoint_path = f'{output_path}/checkpoint/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
    os.mkdir(images_path)
    os.mkdir(meta_path)
    os.mkdir(model_path)
    os.mkdir(checkpoint_path)

meta_data = {
    'model_type': model_type,
    'epochs': epochs,
    'filter_num': filter_num,
    'dependency': dependency_model,
    'patiance': patiance,
    'dataset': {
        'type': dataset_type,
        'image_shape': image_gen_shape,
        'split_size':{
            'train': len(train_gen.masks),
            'val': len(val_gen.masks),
            'test': len(test_gen.masks),
        },
        'augmentation': augment,
    },
    'loss_function':{
        'type': loss_type,
        'parameters': None,
    }
}

json.dump(meta_data, open(f'{meta_path}meta_data.json', 'w'))

# Callbacks            
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=patiance, # Should be more than 10!
    restore_best_weights=True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    mode='min'
)

images, masks = val_gen.__getitem__(0, augment=False)
image = images[0]
images_inter_checkpoint = ImagesInterCheckpoint(image, every_n_epoch=10)
callbacks = [early_stopping, images_inter_checkpoint, checkpoint]
# callbacks = [early_stopping]

Loading images from NRRD format and resizing
Finished loading
Loading images from NRRD format and resizing
Finished loading
Loading images from NRRD format and resizing
Finished loading


In [ ]:
# Train with multiple GPUs :)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
    
    # Metrics
    # mean_iou = tf.keras.metrics.MeanIoU(num_classes=2)
    metrics = ['accuracy']
    
    model = get_model(model_type, image_input_shape, filter_num)
    # loss_func = get_loss_function(loss_type=loss_type)
    # model.compile(optimizer=keras.optimizers.SGD(learning_rate=5e-2), loss=keras.losses.BinaryCrossentropy(), metrics=metrics)
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=5e-2), loss=LocalTversky(), metrics=metrics)
    # model.compile(optimizer=keras.optimizers.Adam(), loss=loss_func, metrics=metrics)
    history = model.fit(train_gen, validation_data=val_gen, epochs=epochs, callbacks=callbacks)
    
# Save progress
model.save(f'{model_path}model')
history_dict = history.history
json.dump(history_dict, open(f'{meta_path}history.json', 'w'))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
Epoch 1/5000
INFO:tensorflow:batch_all_reduce: 382 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 382 all-reduces with algorithm = nccl, num_packs = 1


In [1]:
import keras.backend as K

In [2]:
epsilon = 1e-5
smooth = 1
def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)

def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 4/3
    return K.pow((1-pt_1), gamma)